In [1]:
import httpx
import chardet
from faker import Faker
from bs4 import BeautifulSoup
import datetime
import re

def autodetect(content):
    return chardet.detect(content).get("encoding")

client = httpx.Client(default_encoding=autodetect)
header = Faker().user_agent()



In [2]:
response = client.get(
    r"https://ic.tpex.org.tw/introduce.php",
    headers={"User-Agent": header},
)
response.text

'<!DOCTYPE HTML>\r\n<html>\r\n<head>\r\n<meta http-equiv="Content-Type" content="text/html; charset=utf-8">\r\n<meta http-equiv="X-UA-Compatible" content="IE=Edge">\r\n<meta name="viewport" content="width=device-width, initial-scale=1.0" />\r\n<title> 產業價值鏈資訊平台 > 太陽能產業產業鏈簡介 </title>\r\n<link rel="icon" type="image/ico" href="/images/favicon.ico">\r\n<link rel="SHORTCUT ICON" href="/images/favicon.ico">\r\n<link rel="stylesheet" type="text/css" href="/css/bootstrap/bootstrap.min.css">\r\n<link rel="stylesheet" type="text/css" href="/css/jquery-ui-1.12.1.min.css">\r\n<link rel="stylesheet" type="text/css" href="/css/ic.css">\r\n<script type="text/javascript" src="/js/jquery-3.6.0.min.js"></script>\r\n<script type="text/javascript" src="/js/jquery-ui-1.12.1.min.js"></script>\r\n<script type="text/javascript" src="/js/jquery-ui.catcomplete.js"></script>\r\n<script type="text/javascript" src="/js/public.js"></script>\r\n<script type="text/javascript" src="/js/stock_choice.js"></script>\r\n<

In [3]:
response = client.get(
    r"https://ic.tpex.org.tw/introduce.php",
    headers={"User-Agent": header},
)
soup = BeautifulSoup(response.text, 'html.parser')
all_inds = {ind['value']:ind.text.strip() for ind in soup.find(id='ic_option').find_all('option')}

In [4]:
all_inds

{'D000': '半導體',
 'C100': '製藥',
 'C200': '醫療器材',
 'C300': '食品生技',
 'C400': '再生醫療',
 'A300': '電動車輛產業',
 'A200': 'LED照明產業',
 'A100': '太陽能產業',
 'AB10': '汽電共生',
 'AB20': '風力發電',
 'E000': '能源元件',
 'AD10': '智慧電網',
 '5100': '區塊鏈',
 '5200': '金融科技',
 '5300': '人工智慧',
 '5400': '雲端運算',
 '5500': '資通訊安全',
 '5600': '大數據',
 '5700': '體驗科技',
 '5800': '運動科技',
 '4100': '太空衛星科技',
 'B000': '休閒娛樂',
 'L000': '印刷電路板',
 'R300': '電子商務',
 'J000': '被動元件',
 'I000': '通信網路',
 'K000': '連接器',
 'F000': '電腦及週邊設備',
 'G000': '平面顯示器',
 'H000': '觸控面板',
 '1000': '水泥',
 'M000': '食品',
 'N000': '石化及塑橡膠',
 'O000': '紡織',
 'P000': '電機機械',
 '2000': '造紙',
 'Q000': '鋼鐵',
 '3000': '汽車',
 'R000': '軟體服務',
 'S000': '建材營造',
 'T000': '交通運輸及航運',
 'U000': '金融',
 'V000': '貿易百貨',
 'W000': '油電燃氣',
 'Y000': '文化創意業',
 'X000': '其他'}

In [5]:
import httpx
import chardet
from faker import Faker
from bs4 import BeautifulSoup
import datetime
import re

def autodetect(content):
    return chardet.detect(content).get("encoding")

client = httpx.Client(default_encoding=autodetect)
header = Faker().user_agent()

response = client.get(
    r"https://ic.tpex.org.tw/introduce.php",
    headers={"User-Agent": header},
)
soup = BeautifulSoup(response.text, 'html.parser')
all_inds = {ind['value']:ind.text.strip() for ind in soup.find(id='ic_option').find_all('option')}

all_chains = []
all_comps = []

for main_ind_code in all_inds.keys():
    response = client.get(
        rf"https://ic.tpex.org.tw/introduce.php?ic={main_ind_code}",
        headers={"User-Agent": header},
    )

    soup = BeautifulSoup(response.text, 'html.parser')

    try:
        # Extract subchains
        chains_data = []
        chains = soup.find_all(class_='chain')

        for chain in chains:
            for comp_groups in chain.find_all(class_='chain-company'):
                for comp in comp_groups.find_all(class_='company-chain-panel'):
                    chain_data = {}
                    panel_text = chain.find(class_='chain-title-panel')
                    steam = panel_text.text.strip() if panel_text else ''
                    chain_data['MainIndCode'] = main_ind_code
                    chain_data['MainIndName'] = all_inds[main_ind_code]
                    chain_data['stream'] = steam
                    chain_data['IndCode'] = comp['id'].split('_')[-1]
                    chain_data['IndName']=comp.text.strip()
                    chains_data.append(chain_data)

        print(chains_data)

        sub_inds = {}
        comps = []
        for chain in chains_data:
            ind_code = chain['IndCode']
            print(ind_code)
            sc_ind_pnl = soup.find(id=f'sc-ind-pnl_{ind_code}')
            if sc_ind_pnl:
                for subchain in sc_ind_pnl.find_all(class_='subchain'):
                    sub_inds[subchain['id'].split('_')[-1]] = subchain.text.replace('\xa0','').strip().split('(')[0]
                
                for sub_ind in sub_inds.keys():
                    for comp in soup.find(id=f'sc_company_{sub_ind}').find_all(class_='company-text-over'):
                        if comp.get('href') and re.search(r'\d{4}$',comp['href']):
                            comp_info = {}
                            comp_info['IndCode'] = ind_code
                            comp_info['SubInd'] = sub_inds[sub_ind]
                            comp_info['SubIndCode'] = sub_ind
                            comp_info['Code'] = comp['href'][-4:]
                            comp_info['Name'] = comp.text.strip()
                            comps.append(comp_info)
            else:
                company_list = soup.find(id=f"companyList_{ind_code}")
                if not company_list:
                    continue

                for comp in company_list.find_all(class_='company-text-over'):
                    if comp.get('href') and re.search(r'\d{4}$',comp['href']):
                        comp_info = {}
                        comp_info['IndCode'] = ind_code
                        comp_info['SubIndCode'] = None
                        comp_info['SubInd'] = None
                        comp_info['Code'] = comp['href'][-4:]
                        comp_info['Name'] = comp.text.strip()
                        comps.append(comp_info)
    except Exception as e:
        response.close()
        raise ValueError(e)
    
    all_chains.extend(chains_data)
    all_comps.extend(comps)
        

comp_info



[{'MainIndCode': 'D000', 'MainIndName': '半導體', 'stream': '上游', 'IndCode': 'DC00', 'IndName': 'IP設計/IC設計代工服務'}, {'MainIndCode': 'D000', 'MainIndName': '半導體', 'stream': '上游', 'IndCode': 'D100', 'IndName': 'IC設計'}, {'MainIndCode': 'D000', 'MainIndName': '半導體', 'stream': '中游', 'IndCode': 'D300', 'IndName': 'IC/晶圓製造'}, {'MainIndCode': 'D000', 'MainIndName': '半導體', 'stream': '中游', 'IndCode': 'D400', 'IndName': '生產製程及檢測設備'}, {'MainIndCode': 'D000', 'MainIndName': '半導體', 'stream': '中游', 'IndCode': 'D200', 'IndName': '光罩'}, {'MainIndCode': 'D000', 'MainIndName': '半導體', 'stream': '中游', 'IndCode': 'D500', 'IndName': '化學品'}, {'MainIndCode': 'D000', 'MainIndName': '半導體', 'stream': '下游', 'IndCode': 'D900', 'IndName': 'IC封裝測試'}, {'MainIndCode': 'D000', 'MainIndName': '半導體', 'stream': '下游', 'IndCode': 'D600', 'IndName': '生產製程及檢測設備'}, {'MainIndCode': 'D000', 'MainIndName': '半導體', 'stream': '下游', 'IndCode': 'D700', 'IndName': '基板'}, {'MainIndCode': 'D000', 'MainIndName': '半導體', 'stream': '下游', 'IndCode'

{'IndCode': 'V300',
 'SubIndCode': None,
 'SubInd': None,
 'Code': '7545',
 'Name': '台邦企業'}

In [6]:
all_comps

[{'IndCode': 'DC00',
  'SubIndCode': None,
  'SubInd': None,
  'Code': '2401',
  'Name': '凌陽'},
 {'IndCode': 'DC00',
  'SubIndCode': None,
  'SubInd': None,
  'Code': '3035',
  'Name': '智原'},
 {'IndCode': 'DC00',
  'SubIndCode': None,
  'SubInd': None,
  'Code': '3443',
  'Name': '創意'},
 {'IndCode': 'DC00',
  'SubIndCode': None,
  'SubInd': None,
  'Code': '5222',
  'Name': '全訊'},
 {'IndCode': 'DC00',
  'SubIndCode': None,
  'SubInd': None,
  'Code': '6533',
  'Name': '晶心科'},
 {'IndCode': 'DC00',
  'SubIndCode': None,
  'SubInd': None,
  'Code': '3661',
  'Name': '世芯-KY'},
 {'IndCode': 'DC00',
  'SubIndCode': None,
  'SubInd': None,
  'Code': '3529',
  'Name': '力旺'},
 {'IndCode': 'DC00',
  'SubIndCode': None,
  'SubInd': None,
  'Code': '6643',
  'Name': 'M31'},
 {'IndCode': 'DC00',
  'SubIndCode': None,
  'SubInd': None,
  'Code': '8227',
  'Name': '巨有科技'},
 {'IndCode': 'DC00',
  'SubIndCode': None,
  'SubInd': None,
  'Code': '8299',
  'Name': '群聯'},
 {'IndCode': 'DC00',
  'SubIndCod

In [79]:
soup

<!DOCTYPE HTML>

<html>
<head>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<meta content="IE=Edge" http-equiv="X-UA-Compatible"/>
<meta content="width=device-width, initial-scale=1.0" name="viewport">
<title> 產業價值鏈資訊平台 &gt; 體驗科技產業鏈簡介 </title>
<link href="/images/favicon.ico" rel="icon" type="image/ico"/>
<link href="/images/favicon.ico" rel="SHORTCUT ICON"/>
<link href="/css/bootstrap/bootstrap.min.css" rel="stylesheet" type="text/css"/>
<link href="/css/jquery-ui-1.12.1.min.css" rel="stylesheet" type="text/css"/>
<link href="/css/ic.css" rel="stylesheet" type="text/css"/>
<script src="/js/jquery-3.6.0.min.js" type="text/javascript"></script>
<script src="/js/jquery-ui-1.12.1.min.js" type="text/javascript"></script>
<script src="/js/jquery-ui.catcomplete.js" type="text/javascript"></script>
<script src="/js/public.js" type="text/javascript"></script>
<script src="/js/stock_choice.js" type="text/javascript"></script>
<script src="/js/intro.js" type="text/javascr

In [8]:
import pandas as pd

all_inds_df = pd.DataFrame(all_chains)
all_comps_df = pd.DataFrame(all_comps)

In [11]:
all_comps_data = all_comps_df.merge(all_inds_df,on=['IndCode'],how='left')

In [12]:
all_comps_data

,IndCode,SubIndCode,SubInd,Code,Name,MainIndCode,MainIndName,stream,IndName
0,DC00,None,None,2401,凌陽,D000,半導體,上游,IP設計/IC設計代工服務
1,DC00,None,None,3035,智原,D000,半導體,上游,IP設計/IC設計代工服務
2,DC00,None,None,3443,創意,D000,半導體,上游,IP設計/IC設計代工服務
3,DC00,None,None,5222,全訊,D000,半導體,上游,IP設計/IC設計代工服務
4,DC00,None,None,6533,晶心科,D000,半導體,上游,IP設計/IC設計代工服務
...,...,...,...,...,...,...,...,...,...
6076,V300,None,None,7412,生態綠,V000,貿易百貨,下游,零售通路
6077,V300,None,None,7491,車之輪,V000,貿易百貨,下游,零售通路
6078,V300,None,None,7534,安博森,V000,貿易百貨,下游,零售通路
6079,V300,None,None,7542,宥盛,V000,貿易百貨,下游,零售通路


In [95]:
all_chains

[{'stream': '上游', 'ind_code': 'DC00', 'ind_name': 'IP設計/IC設計代工服務'},
 {'stream': '上游', 'ind_code': 'D100', 'ind_name': 'IC設計'},
 {'stream': '中游', 'ind_code': 'D300', 'ind_name': 'IC/晶圓製造'},
 {'stream': '中游', 'ind_code': 'D400', 'ind_name': '生產製程及檢測設備'},
 {'stream': '中游', 'ind_code': 'D200', 'ind_name': '光罩'},
 {'stream': '中游', 'ind_code': 'D500', 'ind_name': '化學品'},
 {'stream': '下游', 'ind_code': 'D900', 'ind_name': 'IC封裝測試'},
 {'stream': '下游', 'ind_code': 'D600', 'ind_name': '生產製程及檢測設備'},
 {'stream': '下游', 'ind_code': 'D700', 'ind_name': '基板'},
 {'stream': '下游', 'ind_code': 'D800', 'ind_name': '導線架'},
 {'stream': '下游', 'ind_code': 'DA00', 'ind_name': 'IC模組'},
 {'stream': '下游', 'ind_code': 'DB00', 'ind_name': 'IC通路'},
 {'stream': '上游', 'ind_code': 'C110', 'ind_name': '原材料'},
 {'stream': '中游', 'ind_code': 'C120', 'ind_name': '中藥材、西藥原料藥'},
 {'stream': '下游', 'ind_code': 'C130', 'ind_name': '中、西藥製劑'},
 {'stream': '下游', 'ind_code': 'C140', 'ind_name': '藥品代理銷售及通路'},
 {'stream': '下游', 'ind_code

In [39]:
from typing import Dict
import re
import datetime
import chardet

from faker import Faker
from bs4 import BeautifulSoup
import httpx
import pandas as pd


def autodetect(content):
    return chardet.detect(content).get("encoding")

class BaseClient:
    def __init__(self):
        self.client = httpx.Client(default_encoding=autodetect)
        self.header = Faker().user_agent()

    def _parse_html(self,url):
        response = self.client.get(url, headers={"User-Agent": self.header})
        self.check_status_code(response)
        return BeautifulSoup(response.text, "html.parser")

    @staticmethod
    def check_status_code(response):
        if response.status_code != 200:
            raise ValueError("Fail to get the target url")

class IndScraper(BaseClient):
    BASE_URL = "https://ic.tpex.org.tw/introduce.php"

    def __init__(self):
        super().__init__()
        self._industry_data =  self._extract_industries()
        self._chains_data = None

    def scrape_data(self):
        all_chains_df,all_comps_df = self._get_all_chains_data()
        all_inds_comp_df = all_comps_df.merge(all_chains_df,on=['IndCode'],how='left')
        return all_inds_comp_df      

    def _get_all_chains_data(self):
        # Get all industries data
        all_chains_data = []
        all_comps_data = []
        for industry_code in self._industry_data.keys():
            soup = self._parse_html(f'{self.BASE_URL}?ic={industry_code}')
            try:
                
                chains_data,comps_data = self._extract_chains_and_companies(soup, industry_code)
                all_chains_data.extend(chains_data)
                all_comps_data.extend(comps_data)
            except Exception as e:
                raise ValueError(e)

        return pd.DataFrame(all_chains_data),pd.DataFrame(all_comps_data)

    def _extract_chains_and_companies(self, soup, industry_code):
        # Set the data foramt as a list of dictionaries
        
        chain_data_list = []
        company_data_list = []

        chains = soup.find_all(class_='chain')
        chain_data={}
        chain_data['MainIndCode'] = industry_code
        chain_data['MainIndName'] = self._industry_data[industry_code]

        if chains:
            for chain in chains:
                chain_data_list.extend(self._extract_chain_industries(chain_data,chain))
        else:
            chain_data_list.extend(self._extract_chain_industries(chain_data,soup))

        for chain_data in chain_data_list:
            company_data_list.extend(self._extract_companies(soup,chain_data))

        return chain_data_list,company_data_list

    @staticmethod
    def _extract_chain_industries(chain_data,chain,discrete=False):
        all_ind_data =[]
        for comp_groups in chain.find_all(class_='chain-company'):
            for comp in comp_groups.find_all(class_='company-chain-panel'):
                ind_data = chain_data.copy()
                ind_data['IndCode'] = comp['id'].split('_')[-1]
                ind_data['IndName'] = comp.text.strip()
                all_ind_data.append(ind_data)
        return all_ind_data

    @staticmethod
    def _extract_stream(chain):
        # Extract which stream the subindustry is in
        panel_text = chain.find(class_='chain-title-panel')
        return panel_text.text.strip() if panel_text else ''

    def _extract_companies(self,soup, chain):
        """ There are 2 situations:
        1. There is no subindustry, we can just parse from its ordinary industry
        2. There is subindustry, we need to loop into the panel, and then parse from its subindusry
        """
        comps = []
        ind_code = chain['IndCode']
        sc_ind_pnl = soup.find(id=f'sc-ind-pnl_{ind_code}')

        if sc_ind_pnl:
            sub_inds = self._extract_sub_industries(sc_ind_pnl)
            for sub_ind in sub_inds.keys():
                comps.extend(self._get_company_list(soup.find(id=f'sc_company_{sub_ind}'),ind_code,sub_inds[sub_ind],sub_ind))

        else:
            company_list = soup.find(id=f"companyList_{ind_code}")
            if not company_list:
                return []
            comps.extend(self._get_company_list(company_list,ind_code))
        
        return comps


    def _get_company_list(self,company_list_soup,ind_code, sub_ind=None, sub_ind_code=None):
        comps = []
        for comp in company_list_soup.find_all(class_='company-text-over'):
            comp_info = self._get_company_info(comp,ind_code,sub_ind,sub_ind_code)
            if comp_info:
                comps.append(comp_info)
        
        return comps
    
    @staticmethod
    def _get_company_info(comp, ind_code, sub_ind=None, sub_ind_code=None):
        """Helper function to create a dictionary with company information"""
        if comp.get('href') and re.search(r'\d{4}$',comp['href']):
            return {
                'IndCode': ind_code,
                'SubInd': sub_ind,
                'SubIndCode': sub_ind_code,
                'Code': comp['href'][-4:],
                'Name': comp.text.strip()
            }

    @staticmethod    
    def _extract_sub_industries(sc_ind_pnl):
        return {subchain['id'].split('_')[-1]:subchain.text.split('(')[0].replace('\xa0','') for subchain in sc_ind_pnl.find_all(class_=re.compile(r'^subchain'))}

    def _extract_industries(self):
        soup = self._parse_html(self.BASE_URL)
        return {ind['value']: ind.text.strip() for ind in soup.find(id='ic_option').find_all('option')}




In [38]:
ind_scraper = IndScraper()
df = ind_scraper.scrape_data()


{'D110': 'LED驅動IC', 'D120': '光通訊IC', 'D130': '光源管理IC', 'D140': '消費性IC', 'D150': '記憶體IC', 'D160': '記憶體控制IC', 'D170': '微控制器IC', 'D180': '電源管理IC', 'D190': '磁碟儲存控制器IC', 'D1A0': '網路通訊IC', 'D1B0': '輸出入介面IC', 'D1C0': '平面顯示器控制IC', 'D1D0': '平面顯示器驅動IC', 'D1E0': '光儲存控制IC', 'D1F0': '影像感測IC'}
{'D310': '晶圓製造', 'D320': 'DRAM製造', 'D330': '其他IC/二極體製造'}
{'C211': '塑化材料', 'C212': '金屬零件'}
{'C221': '體溫計、血壓計', 'C222': '人工關節', 'C223': '隱型眼鏡、光學鏡片', 'C224': '衛生用品', 'C225': '醫療耗材', 'C226': '生物檢測', 'C227': '顯示器', 'C228': '其他'}
{'C231': '醫療器材代理', 'C232': '醫療院所、醫療器材專賣店、藥局等通路'}
{'5411': '伺服器', '5412': '儲存', '5413': '網路'}
{'5491': 'IaaS', '5492': 'PaaS', '5493': 'SaaS'}
{'5711': 'CPU', '5712': 'GPU', '5713': 'HPU', '5714': 'SoC', '5715': 'IC'}
{'5721': '顯示技術', '5722': '光學元件/模組', '5723': 'LCoS', '5724': 'OLED', '5725': 'Micro LED', '5726': '微型顯示器/光機引擎'}
{'5731': '環境感知', '5732': '深度感測', '5733': 'SLAM', '5734': '其他', '5735': '使用者感知', '5736': '視覺', '5737': '語音', '5738': '手勢/動作', '5739': '觸覺'}
{'5741': '電池模組', '5742': '主機

In [41]:
df

,IndCode,SubInd,SubIndCode,Code,Name,MainIndCode,MainIndName,IndName
0,DC00,None,None,2401,凌陽,D000,半導體,IP設計/IC設計代工服務
1,DC00,None,None,3035,智原,D000,半導體,IP設計/IC設計代工服務
2,DC00,None,None,3443,創意,D000,半導體,IP設計/IC設計代工服務
3,DC00,None,None,5222,全訊,D000,半導體,IP設計/IC設計代工服務
4,DC00,None,None,6533,晶心科,D000,半導體,IP設計/IC設計代工服務
...,...,...,...,...,...,...,...,...
6103,X400,None,None,7614,市民永續,X000,其他,其他
6104,X400,None,None,7619,李連資源,X000,其他,其他
6105,X400,None,None,7620,帝天,X000,其他,其他
6106,X400,None,None,7633,大鼎冷鏈,X000,其他,其他


In [34]:
df1

,MainIndCode,MainIndName,IndCode,IndName
0,D000,半導體,DC00,IP設計/IC設計代工服務
1,D000,半導體,D100,IC設計
2,D000,半導體,D300,IC/晶圓製造
3,D000,半導體,D400,生產製程及檢測設備
4,D000,半導體,D200,光罩
...,...,...,...,...
418,X000,其他,X200,環保潔能服務產業
419,X000,其他,X300,其他電子產品及電子服務產業
420,X000,其他,X500,特殊照明業
421,X000,其他,X600,農業科技業


In [32]:
def remove(l):
    return [dict(t) for t in {tuple(sorted(d.items())) for d in l}]

print(len(ind_scraper._industry_data))
print('all_chain',len(df1))
print('all_ccomps',len(df2))

46
all_chain 423
all_ccomps 6108


In [1]:
import time

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager



def get_csv(url, tag_name):
    driver.get(url)
    time.sleep(2)

    wait = WebDriverWait(driver, 600)
    xpath = f"//{tag_name}[@class='csv']"
    wait.until(EC.presence_of_element_located((By.XPATH, xpath)))
    element = driver.find_element(By.XPATH, xpath)
    element.send_keys("\n")
    time.sleep(2)


chromedriver_path = "chromedriver.exe"
#driver = webdriver.Chrome(executable_path=chromedriver_path)
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
try:
    url_sets = [
        "https://www.twse.com.tw/pcversion/zh/page/trading/fund/TWT38U.html",
        "https://www.twse.com.tw/zh/trading/foreign/twt44u.html",
        "https://www.twse.com.tw/zh/trading/foreign/twt43u.html",
    ]
    tag_names = ["a", "button", "button"]
    for url, tag in zip(url_sets, tag_names):
        get_csv(url, tag)

except Exception as e:
    print(e)

driver.close()


In [4]:
%pip install -U selenium

     ---------------------------------------- 7.2/7.2 MB 12.1 MB/s eta 0:00:00
  Attempting uninstall: selenium
    Found existing installation: selenium 4.8.2
    Uninstalling selenium-4.8.2:
      Successfully uninstalled selenium-4.8.2
Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\s3309\Website\Allium Tuberosum\AT_WEB\Scripts\python.exe -m pip install --upgrade pip' command.


In [3]:
from datetime import datetime,date
import os

import pandas as pd
def get_investor_type_and_date(filename):
    name_split = filename.split('_')
    return name_split[-3:],pd.to_datetime(name_split[1])

def get_rename_cols_and_skiprows(investor_type):

    common_dict = {
    '證券代號':'stock_id',
    '證券名稱':'stock_name',
    }

    if investor_type =='38U':
        common_dict.update(
            {
                '買賣超股數': 'ForeignInvestor',
                '買賣超股數.1':'ForeignDealer_self'
            }
        )
        return common_dict,2
    elif investor_type =='43U':
        common_dict.update(
            {
                '買賣超股數': 'Dealer_self',
                '買賣超股數.1':'DealerHedging'
            }
        )
        return common_dict,2
    elif investor_type =='44U':
        common_dict.update(
            {
                '買賣超股數': 'InvestmentTrust'
            }
        )
        return common_dict,1

def clean_df(df,renamed_cols_mapping):
    renamed_cols = list(renamed_cols_mapping.values())
    df.rename(columns=renamed_cols_mapping)
    df = df.dropna(subset=['stock_name'])
    df['stock_id']=df['stock_id'].replace(r'\D','',regex=True)
    df[renamed_cols]=df[renamed_cols].replace(r',','',regex=True).astype(int)
    return df


def get_institutional_investor_df(download_foler,filename):
    investor_type, date = get_investor_type_and_date(filename)
    rename_cols_mapping,skiprows = get_rename_cols_and_skiprows(investor_type)

    df = pd.read_csv(os.path.join(download_foler,file_name),
                     encoding='big5hkscs',skiprows=skiprows)
    clean_df = clean_df(df,rename_cols_mapping)
    clean_df['date']=date
    return clean_df

def ha():

    today = date.today()

    download_foler = r'C:\Users\s3309\Downloads'
    all_data = pd.DataFrame()

    all_files = os.listdir()
    for file_name in all_files:
        file_path = os.path.join(download_foler,file_name)
        last_modified_timestamp = os.path.getmtime(file_path)
        last_modified_date = datetime.fromtimestamp(last_modified_timestamp).date()
        if last_modified_date== today:
            return
        

file_name = r'C:\Users\s3309\Downloads\TWT44U_20230721.csv'
iv = pd.read_csv(file_name,encoding='big5hkscs',skiprows=1)
date_str = file_name.split('_')[1].split('.')[0]
date = pd.to_datetime(date_str)
renamed_cols_mapping = {
    '證券代號':'stock_id',
    '證券名稱':'stock_name',
    '買賣超股數': 'Dealer_self',
}
iv=iv.dropna(subset=['證券名稱'])
iv.rename(columns=renamed_cols_mapping,inplace=True)

renamed_cols = list(renamed_cols_mapping.values())
iv[renamed_cols].fillna('0',inplace=True)
require_cols =renamed_cols

iv['stock_id']=iv['stock_id'].replace(r'\D','',regex=True)
print(require_cols)
#iv[require_cols]=iv[require_cols].replace(r',','',regex=True).astype(int)

iv[require_cols]

['stock_id', 'stock_name', 'Dealer_self']


C:\Users\s3309\AppData\Local\Temp\ipykernel_30956\3862176268.py:74: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iv[renamed_cols].fillna('0',inplace=True)


,stock_id,stock_name,Dealer_self
0,2890,永豐金,"15,650,000"
1,2891,中信金,"11,613,000"
2,2002,中鋼,"10,074,000"
3,2886,兆豐金,"5,881,000"
4,2382,廣達,"5,365,000"
...,...,...,...
329,1612,宏泰,0
330,2354,鴻準,0
331,2492,華新科,0
332,6278,台表科,0


In [42]:
date_str = datetime.datetime.strftime(datetime.datetime.today(),'%Y%m%d')
date_str

'20230722'

In [2]:
import pandas as pd

news = pd.read_html(r'https://www.twse.com.tw/pcversion/zh/page/trading/fund/TWT38U.html')

ValueError: No tables found matching pattern '.+'

: 

In [17]:
news['發言日期']=news['發言日期'].astype(str).replace(r'^112','2023',regex=True)

news['發言日期'] = pd.to_datetime(news['發言日期'])
news['test'] = news['發言日期']+news['發言時間'].astype(int)
news

TypeError: Addition/subtraction of integers and integer-arrays with DatetimeArray is no longer supported.  Instead of adding/subtracting `n`, use `n * obj.freq`